# MNIST Dataset Processing and Neural Network

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler


2024-12-28 00:11:12.907197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-28 00:11:12.909323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-28 00:11:12.918471: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-28 00:11:12.944137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735344672.980653    1820 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735344672.99

## Step 1: Download and Process MNIST Dataset

In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 16x16 (256 pixels)
x_train = x_train[:, ::2, ::2]  # Downsample by skipping every other pixel
x_test = x_test[:, ::2, ::2]

# Normalize pixel values to range [0.0, 1.0] (0.0 for white, 1.0 for black)
x_train = 1 - x_train / 255.0
x_test = 1 - x_test / 255.0

# Flatten images into 256-pixel vectors
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step 


## Step 2: Save Processed Data to CSV Files

In [3]:
# Save input data to input.csv
input_df = pd.DataFrame(x_train_flat)
input_df.to_csv("input.csv", index=False, header=False)


# Save target data to target.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False, header=False)

print("Saved input.csv and target.csv.")

Saved input.csv and target.csv.


## Step 2.1 Install Rust machine learning library

In [1]:
# clone github.com/mpohl100/ml_rust and cargo build --release

import os
import subprocess

def clone_and_build(repo_url, branch="main"):
    """
    Clones a GitHub repository and runs `cargo build --release`.

    Args:
        repo_url (str): The GitHub repository URL to clone.
        branch (str): The branch to checkout. Default is "main".
    """
    try:
        # Extract repository name from URL
        repo_name = repo_url.split("/")[-1].replace(".git", "")
        
        # Check if the repository already exists
        if not os.path.exists(repo_name):
            print(f"Cloning {repo_url} into {os.getcwd()}...")
            subprocess.run(
                ["git", "clone", "--branch", branch, repo_url],
                check=True
            )
        else:
            print(f"Repository {repo_name} already exists. Skipping clone.")

        # Change into the repository directory
        repo_path = os.path.join(os.getcwd(), repo_name)
        os.chdir(repo_path)

        # Run `cargo build --release`
        print("Running `cargo build --release`...")
        subprocess.run(["cargo", "build", "--release"], check=True)

        print("Build completed successfully!")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred while executing a command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Change back to the original directory
        os.chdir("..")

repo_url = "https://github.com/mpohl100/ml_rust.git"
clone_and_build(repo_url)


Cloning https://github.com/mpohl100/ml_rust.git into /workspaces/ml-jupyter-notebook...


Cloning into 'ml_rust'...


Running `cargo build --release`...


    Updating crates.io index
     Locking 96 packages to latest compatible versions
  Downloaded equivalent v1.0.1
  Downloaded is_terminal_polyfill v1.70.1
  Downloaded anstyle-query v1.1.2
  Downloaded unicode-ident v1.0.14
  Downloaded rayon v1.10.0
  Downloaded cfg-if v1.0.0
  Downloaded anstyle-parse v0.2.6
  Downloaded byteorder v1.5.0
  Downloaded anstyle v1.0.10
  Downloaded anstream v0.6.18
  Downloaded clap v4.5.23
  Downloaded clap_builder v4.5.23
  Downloaded dyn-clone v1.0.17
  Downloaded colorchoice v1.0.3
  Downloaded crossbeam-deque v0.8.6
  Downloaded either v1.13.0
  Downloaded clap_lex v0.7.4
  Downloaded clap_derive v4.5.18
  Downloaded crossbeam-epoch v0.9.18
  Downloaded getrandom v0.2.15
  Downloaded rand_chacha v0.3.1
  Downloaded ppv-lite86 v0.2.20
  Downloaded itoa v1.0.14
  Downloaded csv-core v0.1.11
  Downloaded crossbeam-utils v0.8.21
  Downloaded rand_core v0.6.4
  Downloaded serde v1.0.217
  Downloaded heck v0.5.0
  Downloaded utf8parse v0.2.2
  Download

Build completed successfully!


    Finished `release` profile [optimized] target(s) in 10.50s


## Step 2.2 Train neural network based on shape file

In [4]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
shape_file = "nn_shape.yaml"
training_verification_ratio = 0.7
learning_rate = 0.01
epochs = 100
tolerance = 0.1
batch_size = 32

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/train",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--shape-file", shape_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 256 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 10 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }] }
Epoch: 0
Epoch 0: Loss 0.5481011571700998, Accuracy 6.250148813066978%
Epoch: 1
Epoch 1: Loss 0.2688223892946907, Accuracy 28.743541512893163%
Epoch: 2
Epoch 2: Loss 0.22804147341521125, Accuracy 38.939022357675185%
Epoch: 3
Epoch 3: Loss 0.2118171925028597, Accuracy 43.80818590918831%
Epoch: 4


KeyboardInterrupt: 

## Step 3: Create a Neural Network using Rust neural network generator

In [3]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
training_verification_ratio = 0.7
learning_rate = 0.1
epochs = 100
tolerance = 0.1
batch_size = 32
num_generations = 100
log_level = 1
population_size = 4
num_offsprings = 10

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/nn_generator",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
    "--num-generations", str(num_generations),
    "--log-level", str(log_level),
    "--population-size", str(population_size),
    "--num-offsprings", str(num_offsprings)
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Saving model to: ./trained_model with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 256 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 10 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }] }
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 256 }, activation: ActivationData { activation_type: Softmax, temperature: Some(2.5831520557403564) } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 256 }, activation: ActivationData { activation_type: Softmax, temperature: Some(2.5831520557403564) } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 10 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }] }
Epoch: 0
Inputs: 59999 x 196
Targets:

KeyboardInterrupt: 

## Step 4: Evaluate the Model

In [2]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
tolerance = 0.1

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/evaluate",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--tolerance", str(tolerance),
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Accuracy: 0.6767446124102069
Model training completed successfully and saved in: ./trained_model
